In [2]:
import pandas as pd
import numpy as np
import pymc as pm

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")
flipped

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_52424/3063244417.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")


,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.47,8.77,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.46,8.76,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.43,8.75,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.42,8.74,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.40,8.73,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629274,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629275,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629276,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629277,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [5]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_time', 'new_x', 'new_y', 'chip_side',
       'end_time', 'Right_End', 'Left_End', 'Flip'],
      dtype='object')

In [7]:
flipped[['o', 'dir']]

,o,dir
0,265.97,215.94
1,268.32,217.30
2,268.97,222.15
3,270.01,232.30
4,274.36,241.22
...,...,...
1629274,NaN,NaN
1629275,NaN,NaN
1629276,NaN,NaN
1629277,NaN,NaN


In [43]:
frames_per_play = flipped[flipped['pff_positionLinedUp']=='QB'].groupby('uniqueplayId', as_index=False).size().rename(columns={'size':'play_length_frames'})

In [44]:
frames_per_play

,uniqueplayId,play_length_frames
0,202109090097,43
1,202109120056,37
2,202109120163,32
3,202109120288,27
4,202109120578,27
...,...,...
5693,20211025003536,63
5694,20211025003660,33
5695,20211025003684,53
5696,20211025003904,37


In [46]:
flipped = flipped.merge(frames_per_play, on='uniqueplayId', how='left')